In [1]:
from model.UNet import UNet
from model.DeepLabV3Plus import DeepLabV3Plus
from model.HRNet import HRNet

import os

from PIL import Image

import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [3]:
class SegmentationDataset(Dataset):
  def __init__(self, data_type, mode, transform=None):    
    self.mode = mode
    self.transform = transform
    self.input, self.label = [], []

    # 작업할 폴더의 경로
    path = './dataset/{0}s'.format(data_type) 

    self.input_path = os.path.join(path, self.mode, "data")
    self.label_path = os.path.join(path, self.mode, "label")

    # 폴더 안에 있는 데이터 파일들의 이름을 추출
    data_names = [name.split('.')[0] for name in os.listdir(self.input_path)]

    # 데이터 전처리
    for data_name in data_names:
        self.input.append('{0}/{1}.png'.format(self.input_path, data_name))
        self.label.append('{0}/{1}.png'.format(self.label_path, data_name))

  def __len__(self):
    return len(self.input)

  def __getitem__(self, index):
    input = Image.open(self.input[index])
    label = Image.open(self.label[index])
    
    if self.transform:
      input = self.transform(input)
      label = self.transform(label)

    return input, label

In [4]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset   = SegmentationDataset('building', 'train', transform)
test_dataset    = SegmentationDataset('building', 'test',  transform)

In [5]:
# 저장된 모델 가중치 로드
#model.load_state_dict(torch.load(os.path.join(config["output_dir"], config["model_name"])) 

In [6]:
output_dir_path = "./trained"

def train(model, dataset, num_epoch = 10):
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True) 

    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters()) 

    for epoch in range(num_epoch):
        model.train()

        costs = []
        
        for batch in dataloader:
            input, label = batch
            
            input = input.to(device)
            label = label.to(device)

            hypothesis = model(input)
 
            cost = loss_func(hypothesis, label)

            optimizer.zero_grad()
            cost.backward()
            optimizer.step()

            costs.append(cost.item())

        #print("{0}/{1} : {2}".format(epoch, num_epoch, np.mean(costs)))
        
        #torch.save(model.state_dict(), os.path.join(output_dir_path, "{0}_epoch_{1:d}.pt".format(type(model).__class__.__name__, epoch + 1)))


In [7]:
num_epoch = 10

#model1 = UNet().to(device)
model2 = DeepLabV3Plus().to(device)
#model3 = HRNet().to(device)

#train(model1, train_dataset, num_epoch)
train(model2, train_dataset, num_epoch)
#train(model3, train_dataset, num_epoch)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 64 but got size 1 for tensor number 4 in the list.